In [1]:
import csv
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import math

In [7]:
class databaseObj:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)
    
    def get_nearest_nodes(self, miner, maxer):
        with self.driver.session() as session:
            return session.write_transaction(self._get_nearest_nodes, miner, maxer)

    def attach_incident_node(self, miner, maxer):
        with self.driver.session() as session:
            return session.write_transaction(self._attach_incident_node, miner, maxer)

    def create_and_return_greeting(self, miner, maxer):
        with self.driver.session() as session:
            return session.write_transaction(self._create_and_return_greeting, miner, maxer)

    @staticmethod
    def _create_and_return_greeting(tx, miner, maxer):
        result = tx.run("MATCH (g:LOCATION) "
                        "where ID(g)>$miner and ID(g)<$maxer "  
                        "WITH g.x as long, collect(g) AS nodes "
                        "WHERE size(nodes) >  1 "
                        "FOREACH (g in tail(nodes) | Detach DELETE g)", miner=miner, maxer=maxer)
        return None
    
    @staticmethod
    def _get_nearest_nodes(tx, miner, maxer):
        result = tx.run("CYPHER runtime=slotted "
                        "MATCH (l1:LOCATION) "
                        "where ID(l1)>$miner and ID(l1)<$maxer "
                        "CALL { "
                        "with l1 "
                        "match (l1)<-[:OCCURED_AT]-(l2:INCIDENT) "
                        "return sum(l2.severity)/2+1 as summer "
                        "} "
                        "with l1, summer "
                        "set l1.safety_score = summer", miner=miner, maxer=maxer)
        return None

    @staticmethod
    def _attach_incident_node(tx, miner, maxer):
        result = tx.run("CYPHER runtime=slotted "
                        "MATCH (l1:INCIDENT) "
                        "where ID(l1)<$maxer and ID(l1)>$miner "
                        "CALL { "
                        "WITH l1 "
                        "MATCH (l2:LOCATION) "
                        "with l2, (l1.x-l2.x)*(l1.x-l2.x)+(l1.y-l2.y)*(l1.y-l2.y) AS dist "
                        "where dist < 0.00003 "
                        "RETURN l2, dist "
                        "ORDER BY dist ASC LIMIT 10 "
                        "} "
                        "WITH l1,l2, dist "
                        "MERGE (l1)-[n:OCCURED_AT]->(l2) "
                        "ON CREATE SET n.distance = dist", miner=miner, maxer=maxer)
        return None


In [1]:
def lineToDict(key, value):
    key = key.split(',')
    value = value.split(',')
    result = {}
    for id, val in zip(key, value):
        result[id] = val
    return result

def getSquareDistance(coord1, coord2):
    return math.pow(coord1[0]-coord2[0], 2) + math.pow(coord1[1]-coord2[1], 2)

def closestNode(nodes, location):
    closeNodes = {}
    for key, loc in nodes.items():
        closeNodes[key] = getSquareDistance(loc, location)
    return min(nodes, key=closeNodes.get)

In [ ]:
# graph = Graph("neo4j+s://347bdfd6.production-orch-0002.neo4j.io:7687", auth=("neo4j", "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc"))
greeter = databaseObj("neo4j+ssc://347bdfd6.production-orch-0002.neo4j.io:7687", "neo4j", "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc")
incident_ids = []
with open('UCSCDumpUpdated.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter='\n', quotechar='|')
    for i, row in enumerate(reader):
        if i == 0:
            key = (','.join(row))
        elif i < 0:
            # print(i)
            pass
        else:
            line = (','.join(row))
            line = lineToDict(key, line)
            # print(line['Longitude'])
            # id = greeter.get_nearest_nodes(float(line['Longitude']), float(line['Latitude']), int(line['CurrentPriorityKey']), line['IncidentTypeDescription'])
            try:
                id = greeter.get_nearest_nodes(float(line['Longitude']), float(line['Latitude']), int(line['CurrentPriorityKey']), line['IncidentTypeDescription'])
                # if id != None:
                #     incident_id = greeter.attach_incident_node(id, int(line['CurrentPriorityKey']), line['IncidentTypeDescription'])
                #     # incident_ids.append(id)
                #     # print(incident_id)
            except:
                print('failure at',i)
                pass
        # else:
        #     break

In [10]:
greeter = databaseObj("neo4j+ssc://347bdfd6.production-orch-0002.neo4j.io:7687", "neo4j", "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc")
# greeter.create_and_return_greeting()
miner = 0
maxer = miner + 5000

counter = 0
while 219537 > maxer:
    greeter.create_and_return_greeting(miner, maxer)
    miner+=5000
    maxer+=5000
    if counter%10 == 0:
        print(maxer)
    counter+=1

# intersections = greeter.get_nearest_nodes([36.9680,-121.9303], 10)
# print(intersections)
# '''
# CYPHER runtime=slotted
# MATCH (l1:LOCATION)
# CALL {
#     with l1
#     match (l1)-[n:ROAD]-(l2:LOCATION)
#     set n.safety_score = (l1.safety_score+l2.safety_score)*n.length
# }'''

Failed to read from defunct connection IPv4Address(('neo4j-core-347bdfd6-7.production-orch-0002.neo4j.io', 7687)) (IPv4Address(('104.197.20.211', 7687)))


IncompleteCommit: Failed to read from defunct connection IPv4Address(('neo4j-core-347bdfd6-7.production-orch-0002.neo4j.io', 7687)) (IPv4Address(('104.197.20.211', 7687)))

In [15]:
greeter = databaseObj("neo4j+ssc://347bdfd6.production-orch-0002.neo4j.io:7687", "neo4j", "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc")
miner = 0
maxer = miner + 1000
counter = 0
while 219537 > maxer:
    greeter.get_nearest_nodes(miner, maxer)
    miner+=1000
    maxer+=1000
    if counter%10 == 0:
        print(maxer)
    counter+=1

2000
12000
22000
32000
42000
52000
62000
72000
82000
92000
102000
112000
122000
132000
142000
152000
162000
172000
182000
192000
202000
212000


In [5]:
greeter = databaseObj("neo4j+ssc://347bdfd6.production-orch-0002.neo4j.io:7687", "neo4j", "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc")
miner = 493538
maxer = miner + 1000
counter = 0
while 554973 > maxer:
    greeter.attach_incident_node(miner, maxer)
    miner+=1000
    maxer+=1000
    if counter%10 == 0:
        print(maxer)
    counter+=1

495538
505538
515538


Failed to read from defunct connection ResolvedIPv4Address(('104.197.20.211', 7687)) (IPv4Address(('104.197.20.211', 7687)))
Unable to retrieve routing information
Transaction failed and will be retried in 0.9311936220864584s (Unable to retrieve routing information)


525538
535538
545538
555538
